In [45]:
import pyspark
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col, date_trunc,hour,unix_timestamp
from pyspark.sql import types

In [46]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [5]:
df_dataset= spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [6]:
df_dataset.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [7]:
df_dataset.repartition(4).write.parquet('revenue', mode='overwrite')


In [12]:
df_dataset.registerTempTable('trips_table')

In [14]:
df_dataset = df_dataset.withColumn("event_time", col("event_time").cast("timestamp"))


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `event_time` cannot be resolved. Did you mean one of the following? [`VendorID`, `extra`, `mta_tax`, `payment_type`, `Airport_fee`].;
'Project [VendorID#0, tpep_pickup_datetime#1, tpep_dropoff_datetime#2, passenger_count#3L, trip_distance#4, RatecodeID#5L, store_and_fwd_flag#6, PULocationID#7, DOLocationID#8, payment_type#9L, fare_amount#10, extra#11, mta_tax#12, tip_amount#13, tolls_amount#14, improvement_surcharge#15, total_amount#16, congestion_surcharge#17, Airport_fee#18, cast('event_time as timestamp) AS event_time#138]
+- Relation [VendorID#0,tpep_pickup_datetime#1,tpep_dropoff_datetime#2,passenger_count#3L,trip_distance#4,RatecodeID#5L,store_and_fwd_flag#6,PULocationID#7,DOLocationID#8,payment_type#9L,fare_amount#10,extra#11,mta_tax#12,tip_amount#13,tolls_amount#14,improvement_surcharge#15,total_amount#16,congestion_surcharge#17,Airport_fee#18] parquet


In [15]:
spark.sql("""
SELECT 
    count(1)
FROM

    trips_table

WHERE

    date_trunc("day", tpep_pickup_datetime) == "2024-10-15"


""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [18]:
df_dataset.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [19]:
df_dataset.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [17]:
df_dataset.select(col("tpep_pickup_datetime")).show()

+--------------------+
|tpep_pickup_datetime|
+--------------------+
| 2024-10-01 00:30:44|
| 2024-10-01 00:12:20|
| 2024-10-01 00:04:46|
| 2024-10-01 00:12:10|
| 2024-10-01 00:30:22|
| 2024-10-01 00:31:20|
| 2024-10-01 00:42:57|
| 2024-10-01 00:59:55|
| 2024-10-01 00:00:47|
| 2024-10-01 00:17:36|
| 2024-10-01 00:49:00|
| 2024-10-01 00:07:26|
| 2024-10-01 00:17:11|
| 2024-10-01 00:00:34|
| 2024-09-30 23:58:40|
| 2024-10-01 00:55:29|
| 2024-10-01 00:08:59|
| 2024-10-01 00:18:38|
| 2024-10-01 00:18:38|
| 2024-10-01 00:39:33|
+--------------------+
only showing top 20 rows



In [ ]:
spark.sql("""
SELECT 
    MAX(trip_distance)
FROM

    trips_table

WHERE

    date_trunc("day", tpep_pickup_datetime) == "2024-10-15"


""").show()

In [22]:
spark.sql("""
SELECT
    date_trunc("hour", tpep_pickup_datetime) AS hour

FROM

    trips_table


 """).show()
    

+-------------------+
|               hour|
+-------------------+
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-09-30 23:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
|2024-10-01 00:00:00|
+-------------------+
only showing top 20 rows



In [38]:
df = df_dataset.withColumn("pickup_hour", hour(col("tpep_pickup_datetime")))

In [36]:
df.schema


StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True), StructField('pickup_hour',

In [39]:
df_update = df.withColumn("dropoff_hour", hour(col("tpep_dropoff_datetime")))

In [42]:
df_update.select("pickup_hour", "dropoff_hour").show()

+-----------+------------+
|pickup_hour|dropoff_hour|
+-----------+------------+
|          0|           0|
|          0|           0|
|          0|           0|
|          0|           0|
|          0|           0|
|          0|           0|
|          0|           0|
|          0|           1|
|          0|           0|
|          0|           0|
|          0|           0|
|          0|           0|
|          0|           0|
|          0|           0|
|         23|           0|
|          0|           1|
|          0|           0|
|          0|           0|
|          0|           0|
|          0|           0|
+-----------+------------+
only showing top 20 rows



In [43]:
df_update.registerTempTable('trips_table_update')

/opt/spark/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [44]:
spark.sql("""
SELECT 
    MAX(pickup_hour)
FROM

    trips_table_update



""").show()

+----------------+
|max(pickup_hour)|
+----------------+
|              23|
+----------------+



In [51]:
df = df_dataset.withColumn("trip_duration_hours", 
                   (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600)

# longest_trip = df.select(max(col("trip_duration_hours"))).collect()[0][0]
# print(f"Longest trip duration: {longest_trip} hours")

In [56]:
df.select("trip_duration_hours").show()
df.registerTempTable('trips_table_update')


+--------------------+
| trip_duration_hours|
+--------------------+
|               0.295|
| 0.21805555555555556|
| 0.15166666666666667|
| 0.18083333333333335|
|0.004722222222222222|
| 0.07777777777777778|
| 0.10111111111111111|
| 0.04138888888888889|
|0.059722222222222225|
|  0.1461111111111111|
| 0.05555555555555555|
| 0.09833333333333333|
| 0.13027777777777777|
|              0.0775|
| 0.36277777777777775|
|               0.785|
|  0.5330555555555555|
|              0.3025|
|              0.3025|
| 0.24055555555555555|
+--------------------+
only showing top 20 rows



/opt/spark/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [57]:
spark.sql("""
SELECT 
    MAX(trip_duration_hours)
FROM

    trips_table_update



""").show()

+------------------------+
|max(trip_duration_hours)|
+------------------------+
|      162.61777777777777|
+------------------------+



In [58]:
print(spark.sparkContext.uiWebUrl)

http://de-zoomcamp.us-central1-c.c.shareprompt-412612.internal:4041
